In [1]:
# !pip install gekko
from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
def print_results(name, x1, x2, x3, x4, obj):
    print(f'Results for {name}')
    print(f'x1: {x1}')
    print(f'x2: {x2}')
    print(f'x3: {x3}')
    print(f'x4: {x4}')
    print(f'Objective: {obj:.5f}')

Suppose we have 

* $z \in \mathbb{R}^{n_z}$ - continuous variable
* $y \in \mathbb{R}^{n_y}$ - integer variable

So we have **Mixed-Integer Non Linear Programming** problem:

$
\min\limits_{y, z}{F(y, z)}, \quad
\left\{ \begin{array}{l}
G(y, z) = 0 \\
H(y, z) \leq 0 \\
y \in P \\
y \in \mathbb{Z}^{n_y}
\end{array} \right.
$

**Proposal three-step algorithm**:

$
J_{NLP}(y) = \min\limits_{z}{F(y, z)}, \quad
\left\{ \begin{array}{l}
G(y, z) = 0 \\
H(y, z) \leq 0 \\
\end{array} \right.
$

**S1**:
$
(y^*, z^*) = \arg\min\limits_{y}{J_{NLP}(y)}, \quad
y \in P
$

**S2**:
$
y^{**} = \arg\min\limits_{y}{d(y, y^*)}, \quad
y \in P \cup Z^{n_y}
$

**S3**:
$
z^{***} = \arg J_{NLP}(y^{**}) = \arg\min\limits_{z}{F(y^{**}, z)}, \quad
\left\{ \begin{array}{l}
G(y, z) = 0 \\
H(y, z) \leq 0 \\
\end{array} \right.
$

$d(y, y^*) = \|y - y^*\|$

**Example**:

* $x_1, x_2$ - continuous 
* $x_3, x_4$ - integer
* $(x_1, x_2, x_3, x_4)_0 = (1, 5, 5, 1)$

$
\min\limits_{x_1, x_2, x_3, x_4}{x_1 x_4 (x_1 + x_2 + x_3) + x_3}, \quad
\left\{ \begin{array}{l}
x_1^2 + x_2^2 + x_3^2 + x_4^2 = 40 \\
x_1 x_2 x_3 x_4 \geq 25 \\
1 \leq x_1, x_2, x_3, x_4 \leq 5 \\
x_3, x_4 \in \mathbb{Z}
\end{array} \right.
$

It means that:
$
F(\overbrace{x_1, x_2}^{y}, \overbrace{x_3, x_4}^{z}) = 
x_1 x_4 (x_1 + x_2 + x_3) + x_3 \\
G(x_1, x_2, x_3, x_4) = x_1^2 + x_2^2 + x_3^2 + x_4^2 - 40 \\
H(x_1, x_2, x_3, x_4) = 25 - x_1 x_2 x_3 x_4 \\
P = \{x_1, x_2, x_3, x_4 \mid 1 \leq x_1, x_2, x_3, x_4 \leq 5 \}
$

Let solve this by an existing MINLP-solver from GEKKO

In [3]:
m = GEKKO()           # Initialize gekko
m.options.SOLVER = 1  # APOPT is an MINLP solver

# optional solver settings with APOPT
m.solver_options = [
    'minlp_maximum_iterations 500',   # minlp maximum iterations
    'minlp_max_iter_with_int_sol 10', # minlp iterations with integer solution
    'minlp_as_nlp 0',                 # treat minlp as nlp 
    'nlp_maximum_iterations 50',      # nlp sub-problem max iterations
    'minlp_branch_method 1',          # 1 = depth first, 2 = breadth first
    'minlp_integer_tol 0.05',         # maximum deviation from whole number
    'minlp_gap_tol 0.01'              # covergence tolerance
]

# Continuous variables
x1 = m.Var(value=1,lb=1,ub=5)
x2 = m.Var(value=5,lb=1,ub=5)

# Integer variables
x3 = m.Var(value=5, lb=1, ub=5, integer=True)
x4 = m.Var(value=1, lb=1, ub=5, integer=True)

# Constraints: equality 
def G(x1, x2, x3, x4):
    return x1 ** 2 + x2 ** 2 + x3 ** 2 + x4 ** 2 - 40

# Constraints: inequality
def H(x1, x2, x3, x4):
    return 25 - x1 * x2 * x3 * x4

# Objective function
def F(x1, x2, x3, x4):
    return x1 * x4 * (x1 + x2 + x3) + x3

m.Equation(G(x1, x2, x3, x4) == 0) # G function
m.Equation(H(x1, x2, x3, x4) <= 0) # H function

# Objective
m.Obj(F(x1, x2, x3, x4))
m.solve(disp=False)

In [4]:
x1_minlp, x2_minlp, x3_minlp, x4_minlp = x1.value[0], x2.value[0], x3.value[0], x4.value[0]
obj_minlp = m.options.objfcnval

In [5]:
print_results('MINLP', x1_minlp, x2_minlp, x3_minlp, x4_minlp, obj_minlp)

Results for MINLP
x1: 1.3589086474
x2: 4.5992789966
x3: 4.0
x4: 1.0
Objective: 17.53227


Now we divide the solving into steps:

**S1** : solve the relaxed corresponding problem
$
(y^*, z^*) = \arg\min\limits_{y}{J_{NLP}(y)}, \quad
y \in P
$

In [6]:
m = GEKKO()           # Initialize gekko
m.options.SOLVER = 3  # Best solver

# Continuous variables
x1 = m.Var(value=1, lb=1, ub=5)
x2 = m.Var(value=5, lb=1, ub=5)

# Integer variables
x3 = m.Var(value=5, lb=1, ub=5)
x4 = m.Var(value=1, lb=1, ub=5)

# Equations
m.Equation(G(x1, x2, x3, x4) == 0) # G function
m.Equation(H(x1, x2, x3, x4) <= 0) # H function

# Objective
m.Obj(F(x1, x2, x3, x4))
m.solve(disp=False)

In [7]:
x1_nlp, x2_nlp, x3_nlp, x4_nlp = x1.value[0], x2.value[0], x3.value[0], x4.value[0]
obj_nlp = m.options.objfcnval

In [8]:
print_results('NLP', x1_nlp, x2_nlp, x3_nlp, x4_nlp, obj_nlp)

Results for NLP
x1: 1.000000057
x2: 4.74299963
x3: 3.8211500283
x4: 1.3794081795
Objective: 17.01402


**S2**: approximate our continuous solution by an integer one

$
y^{**} = \arg\min\limits_{y}{d(y, y^*)}, \quad
y \in P \cup Z^{n_y}
$

$d(y, y^*) = \|y - y^*\|$

In [9]:
m = GEKKO()          # Initialize gekko
m.options.SOLVER = 1 # APOPT is an MINLP solver

# Continuous variables
x1 = m.Const(value=x1_nlp)
x2 = m.Const(value=x2_nlp)

# Integer variables
x3 = m.Var(value=x3_nlp, lb=1, ub=5, integer=True)
x4 = m.Var(value=x4_nlp, lb=1, ub=5, integer=True)

# Objective distance function
def dist(x1, x2, x3, x4, x1_nlp, x2_nlp, x3_nlp, x4_nlp):
    return (x3 - x3_nlp)**2 + (x4 - x4_nlp)**2
    
# Objective
m.Obj(dist(x1, x2, x3, x4, x1_nlp, x2_nlp, x3_nlp, x4_nlp))
m.solve(disp=False)

In [10]:
x1_dist, x2_dist, x3_dist, x4_dist = x1.value, x2.value, x3.value[0], x4.value[0]
obj_dist = m.options.objfcnval

In [11]:
print_results('DIST', x1_dist, x2_dist, x3_dist, x4_dist, obj_dist)

Results for DIST
x1: 1.000000057
x2: 4.74299963
x3: 4.0
x4: 1.0
Objective: 0.17594


**S3**: find the corresponding $z$

$
z^{***} = \arg J_{NLP}(y^{**}) = \arg\min\limits_{z}{F(y^{**}, z)}, \quad
\left\{ \begin{array}{l}
G(y, z) = 0 \\
H(y, z) \leq 0 \\
\end{array} \right.
$

In [12]:
m = GEKKO()           # Initialize gekko
m.options.SOLVER = 3  # Best solver

# Continuous variables
x1 = m.Var(value=x1_nlp, lb=1, ub=5)
x2 = m.Var(value=x2_nlp, lb=1, ub=5)

# Integer variables
x3 = m.Const(value=x3_dist)
x4 = m.Const(value=x4_dist)

# Equations
m.Equation(G(x1, x2, x3, x4) == 0) # G function
m.Equation(H(x1, x2, x3, x4) <= 0) # H function

# Objective
m.Obj(F(x1, x2, x3, x4))
m.solve(disp=False)

In [13]:
x1_miqp, x2_miqp, x3_miqp, x4_miqp = x1.value[0], x2.value[0], x3.value, x4.value
obj_miqp = m.options.objfcnval

In [14]:
print_results('MIQP', x1_miqp, x2_miqp, x3_miqp, x4_miqp, obj_miqp)

Results for MIQP
x1: 1.3589086528
x2: 4.599278995
x3: 4.0
x4: 1.0
Objective: 17.53227


Measure the results - the minlp solution and the corresponding miqp solution:

In [15]:
print_results('MINLP - MIQP', 
              x1_minlp - x1_miqp, 
              x2_minlp - x2_miqp, 
              x3_minlp - x3_miqp, 
              x4_minlp - x4_miqp, 
              obj_minlp - obj_miqp)

Results for MINLP - MIQP
x1: -5.4000000027087935e-09
x2: 1.6000001323845936e-09
x3: 0.0
x4: 0.0
Objective: -0.00000


### More complex case

In [16]:
# # Функция для отрисовки графиков

# def show_results(results, mv_name, cv_name): 
#     time = results['time']
#     x = results[mv_name]
#     x_ref = results[mv_name+'.tr']
#     u = results[cv_name]
#     obj = results['obj']

#     plt.figure(figsize=(10, 5))
#     plt.suptitle(f'Objective value: {obj:.4f}')
    
#     plt.subplot(2, 1, 1)
#     plt.step(time, u, 'o-', 
#              markersize=5, linewidth=1.4,
#              where='post')
#     plt.ylabel('Control u')

#     plt.subplot(2, 1, 2)
#     plt.plot(time, x_ref, 'k-', 
#              label='Reference trajectory')
#     plt.plot(time, x, '-o', c='r',
#              markersize=5, linewidth=1,
#              label='Predicted trajectory')
#     plt.legend(loc=1)
#     plt.ylabel('State x')
#     plt.xlabel('Time')

#     plt.show()

$
\begin{array}{rll}
\text{continuous time}: & 
    \dot{x}(t) = f_c(x(t), b(t)) = x^3(t) - b(t) \ , & 
    t \in \mathbb{R}_{\geq 0} \\
\text{discrete time}: & 
    x^{+} = f_d(x, b, k) = \text{Runge-Kutta-4}\ (f_c) \ , & 
    k \in \mathbb{Z}_{\geq 0}
\end{array}
$

$
\textbf{x} = (x(0), x(1), \dots, x(N)) \\
\textbf{b} = (b(0), b(1), \dots, b(N-1)) \\
\textbf{b} \in \mathbb{Z}^{N} \cap \mathbb{B}, \; \text{where} \;
\mathbb{Z} = \{0, 1\}, \;
\mathbb{B} = 
\left\{ b \in [0, 1]^N \mid \begin{align} 
    & b(k) \geq b(k-1) - b(k-2) \\
    & b(k) \geq b(k-1) - b(k-3) 
\end{align} \right\}
$

$
\text{objetive function}:
F(\textbf{x}, \textbf{b}) = \frac{1}{2} \sum\limits^{N}_{k=0} (x(k) - x_{ref})^2
$

$
\textbf{MINLP}: 
\min\limits_{\textbf{b}}
F(\textbf{x}, \textbf{b}) \\
\text{such that} \\
x^{+} = f_d(x, b, k) \\
x(0) = x_0 \\
\textbf{b} \in \mathbb{Z}^{N} \cap \mathbb{B} \\
N = 30, \ x_0 = 0.8, \ x_{ref} = 0.7
$

In [17]:
# # Constants
# x_0 = 0.8
# x_ref = 0.7
# N = 30
# h = 0.05

# # Model
# m = GEKKO()
# m.time = np.arange(N + 1) * h

# m.options.SOLVER=1  # APOPT is an MINLP solver

# # optional solver settings with APOPT
# m.solver_options = ['minlp_maximum_iterations 500', \
#                     # minlp iterations with integer solution
#                     'minlp_max_iter_with_int_sol 100', \
#                     # treat minlp as nlp
#                     'minlp_as_nlp 0', \
#                     # nlp sub-problem max iterations
#                     'nlp_maximum_iterations 50', \
#                     # 1 = depth first, 2 = breadth first
#                     'minlp_branch_method 1', \
#                     # maximum deviation from whole number
#                     'minlp_integer_tol 0.05', \
#                     # covergence tolerance
#                     'minlp_gap_tol 0.01']

# # Manipulated variable
# u = m.MV(value=0, integer=True, lb=0, ub=1, name='u')
# u.STATUS = 1  # allow optimizer to change
# u.DCOST = 0.0 # smooth out u movement

# # Controlled Variable
# x = m.CV(value=0.8, name='x')
# x.STATUS = 1  # add the SP to the objective
# m.options.CV_TYPE = 2 # squared error
# x.SP = 0.7     # set point
# x.TR_INIT = 0 # set point trajectory
# # x.TAU = 5     # time constant of trajectory

# # Process model
# m.Equation(x.dt() == x**3 - u)

# m.options.IMODE = 6 # control
# m.solve(disp=False)

# # get additional solution information
# with open(m.path+'//results.json') as f:
#     results = json.load(f)
# results['obj'] = m.options.OBJFCNVAL
    
# show_results(results, 'x', 'int_u')

In [18]:
# m.solver_options = ['minlp_maximum_iterations 500', \
#                     'minlp_max_iter_with_int_sol 100', \
#                     'minlp_as_nlp 1', \
#                     'nlp_maximum_iterations 50', \
#                     'minlp_branch_method 1', \
#                     'minlp_integer_tol 0.05', \
#                     'minlp_gap_tol 0.01']
# m.solve(disp=False)

# with open(m.path+'//results.json') as f:
#     results_nl = json.load(f)
# results_nl['obj'] = m.options.OBJFCNVAL
    
# show_results(results_nl, 'x', 'int_u')

In [19]:
# m.solver_options = ['minlp_maximum_iterations 500', \
#                     'minlp_max_iter_with_int_sol 100', \
#                     'minlp_as_nlp 1', \
#                     'nlp_maximum_iterations 50', \
#                     'minlp_branch_method 1', \
#                     'minlp_integer_tol 0.05', \
#                     'minlp_gap_tol 0.01']
# m.solve(disp=False)

# with open(m.path+'//results.json') as f:
#     results_nl = json.load(f)
# results_nl['obj'] = m.options.OBJFCNVAL
    
# show_results(results_nl, 'x', 'int_u')